# Modelo de Clasificación Jerárquica con Aumento de Datos v3.0

Este notebook implementa un pipeline avanzado de clasificación jerárquica:

1.  **Carga y Preprocesamiento**: Usa `hate_speech_twitter` y realiza limpieza de texto (tokenización, stemming, etc.).
2.  **Generación de Características Dual**: Crea embeddings de BERT y vectores TF-IDF.
3.  **Aumento de Datos Sintéticos**: Utiliza **CTGAN** de la librería `sdv` para generar datos sintéticos y **balancear las sub-categorías** de discurso de odio en el conjunto de entrenamiento, mejorando la robustez del modelo.
4.  **Entrenamiento de Clasificador Principal (Nivel 1)**: Entrena y optimiza un ensemble de cuatro modelos para distinguir entre `odio` y `no-odio`.
5.  **Entrenamiento de Clasificador de Sub-categorías (Nivel 2)**: Entrena un modelo XGBoost para clasificar el **tipo de odio** (ej. sexismo, racismo) en los textos ya identificados como odio.
6.  **Evaluación Jerárquica**: Evalúa el rendimiento del pipeline completo en dos niveles, reportando la precisión tanto en la detección de odio como en la clasificación de su tipo.

## 1. Instalación y Configuración

In [ ]:
#!pip install transformers torch datasets scikit-learn xgboost pandas seaborn matplotlib tqdm optuna nltk scipy sdv nltk

In [ ]:
import pandas as pd
import numpy as np
import os
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import time
import torch
import xgboost as xgb
from tqdm.auto import tqdm
import re

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, log_loss, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('punkt')
    nltk.download('stopwords')

BERT_MODEL_NAME = 'bert-base-uncased'
MAX_SAMPLES = 10000 # Aumentar para un mejor entrenamiento de sub-categorías
MAX_TOKEN_LENGTH = 128

# --- Configuración de Dispositivo (GPU o CPU) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# --- Definición de Rutas Locales ---
job_id = f"hierarchical-job-{int(time.time())}"
BASE_DIR = "datos_locales"
PROCESSED_DIR = os.path.join(BASE_DIR, "processed", job_id)
MODEL_OUTPUT_DIR = os.path.join(BASE_DIR, "model_output", job_id)
os.makedirs(PROCESSED_DIR, exist_ok=True)
os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)
PROCESSED_DATA_PATH = os.path.join(PROCESSED_DIR, "processed_data_with_embeddings.csv")
print(f"\nID de trabajo: {job_id}")

c:\Users\emicr\anaconda3\envs\ml_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Usando dispositivo: cuda

ID de trabajo: hierarchical-job-1750836320


## 2. Carga, Análisis y Preprocesamiento de Datos

In [ ]:
print("Cargando dataset 'thefrankhsu/hate_speech_twitter'...")
dataset = load_dataset("thefrankhsu/hate_speech_twitter")
df = pd.DataFrame(dataset['train'])

# Renombrar columnas y manejar nulos en 'categories'
df = df.rename(columns={'tweet': 'text_raw', 'label': 'main_label', 'categories': 'sub_label_str'})
df['sub_label_str'] = df['sub_label_str'].fillna('not-hate')

if MAX_SAMPLES is not None:
    # Asegurarse de que el tamaño de la muestra no sea mayor que la población
    sample_size = min(MAX_SAMPLES, len(df))
    print(f"Tomando una muestra aleatoria de {sample_size} registros (de un total de {len(df)}).")
    df = df.sample(n=sample_size, random_state=42, replace=False).reset_index(drop=True)

print("Distribución de etiquetas principales:")
print(df['main_label'].value_counts())

print("\nDistribución de sub-etiquetas (solo para 'odio'):")
print(df[df['main_label'] == 1]['sub_label_str'].value_counts())

# Codificar sub-etiquetas
from sklearn.preprocessing import LabelEncoder
sub_label_encoder = LabelEncoder()
df['sub_label_encoded'] = sub_label_encoder.fit_transform(df['sub_label_str'])
sub_label_mapping = dict(zip(sub_label_encoder.classes_, sub_label_encoder.transform(sub_label_encoder.classes_)))
print("\nMapeo de Sub-etiquetas:", sub_label_mapping)

# Preprocesamiento de texto
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
def clean_text(text, apply_stemming=False):
    if pd.isna(text): return ""
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|#','', text)
    tokens = word_tokenize(text)
    words = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
    if apply_stemming: words = [stemmer.stem(word) for word in words]
    return " ".join(words)

tqdm.pandas(desc="Limpiando Texto para Embeddings")
df['text_cleaned'] = df['text_raw'].progress_apply(lambda x: clean_text(x, apply_stemming=False))
tqdm.pandas(desc="Aplicando Stemming para TF-IDF")
df['text_stemmed'] = df['text_cleaned'].progress_apply(lambda x: " ".join([stemmer.stem(word) for word in x.split()]))

Cargando dataset 'thefrankhsu/hate_speech_twitter'...
Tomando una muestra aleatoria de 5679 registros (de un total de 5679).
Distribución de etiquetas principales:
main_label
0    4163
1    1516
Name: count, dtype: int64

Distribución de sub-etiquetas (solo para 'odio'):
sub_label_str
Race                   523
Sexual Orientation     429
Gender                 279
Physical Appearance     73
Religion                52
Behavior                40
Class                   40
Ethnicity               40
Disability              40
Name: count, dtype: int64

Mapeo de Sub-etiquetas: {'Behavior': 0, 'Class': 1, 'Disability': 2, 'Ethnicity': 3, 'Gender': 4, 'Physical Appearance': 5, 'Race': 6, 'Religion': 7, 'Sexual Orientation': 8, 'not-hate': 9}


Aplicando Stemming para TF-IDF: 100%|██████████| 5679/5679 [00:00<00:00, 10888.97it/s]


## 3. Generación de Embeddings y División de Datos

In [4]:
print(f"Cargando modelo y tokenizador BERT: {BERT_MODEL_NAME}")
tokenizer_bert = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
model_bert = AutoModel.from_pretrained(BERT_MODEL_NAME).to(device)
model_bert.eval()

def get_bert_embeddings(batch_text):
    inputs = tokenizer_bert(batch_text, padding=True, truncation=True, max_length=MAX_TOKEN_LENGTH, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model_bert(**inputs)
    return outputs.last_hidden_state[:, 0, :].cpu().numpy()

print("Generando embeddings...")
batch_size = 32
all_embeddings = np.vstack([get_bert_embeddings(df.iloc[i:i+batch_size]['text_cleaned'].tolist()) for i in tqdm(range(0, len(df), batch_size))])

embedding_cols = [f'dim_{i}' for i in range(all_embeddings.shape[1])]
df_embeddings = pd.DataFrame(all_embeddings, columns=embedding_cols, index=df.index)
df_processed = pd.concat([df, df_embeddings], axis=1)

print("\n--- Dividiendo Datos ---")
y_main = df_processed['main_label'].values
df_trainval, df_test = train_test_split(df_processed, test_size=0.2, random_state=42, stratify=y_main)
y_trainval_main = df_trainval['main_label'].values
df_train, df_val = train_test_split(df_trainval, test_size=0.25, random_state=42, stratify=y_trainval_main)

print(f"Tamaño Train: {len(df_train)}, Val: {len(df_val)}, Test: {len(df_test)}")

Cargando modelo y tokenizador BERT: bert-base-uncased
Generando embeddings...


100%|██████████| 178/178 [00:05<00:00, 35.17it/s]


--- Dividiendo Datos ---
Tamaño Train: 3407, Val: 1136, Test: 1136


## 4. Aumento de Datos Sintéticos para Sub-categorías (CTGAN)
Nos enfocamos en el desbalance de las sub-categorías de 'odio'. Usaremos CTGAN para generar nuevos datos de entrenamiento para las clases minoritarias, basándonos en sus embeddings.

In [ ]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
from sklearn.preprocessing import LabelEncoder

print("--- Preparando datos para aumento ---")
# 1. Aislar los datos de entrenamiento que son 'odio'
df_train_hate = df_train[df_train['main_label'] == 1].copy()
features_to_augment = ['sub_label_str'] + embedding_cols
df_to_augment = df_train_hate[features_to_augment]

print("Distribución de sub-categorías ANTES del aumento:")
hate_counts = df_to_augment['sub_label_str'].value_counts()
print(hate_counts)

# Crear un nuevo encoder dedicado SOLO para las sub-categorías de odio.
sub_hate_only_encoder = LabelEncoder()

if len(hate_counts) > 1 and not df_to_augment.empty:
    # Ajustar el nuevo encoder solo con las etiquetas de odio
    sub_hate_only_encoder.fit(df_to_augment['sub_label_str'])
    print("\nNuevo encoder para Nivel 2 creado. Clases:", sub_hate_only_encoder.classes_)
    
    # 2. Configurar metadatos
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data=df_to_augment)
    
    print("\nActualizando metadatos para tratar embeddings como numéricos continuos...")
    for col in embedding_cols:
        metadata.update_column(column_name=col, sdtype='numerical')
    metadata.update_column(column_name='sub_label_str', sdtype='categorical')

    # 3. Configurar y entrenar el sintetizador CTGAN
    use_gpu = torch.cuda.is_available()
    print(f"Usando GPU: {use_gpu}")
    synthesizer = CTGANSynthesizer(
        metadata, 
        epochs=150, 
        embedding_dim=64, 
        verbose=False,
        cuda=use_gpu  
    )
    
    print(f"\nEntrenando CTGAN para generar datos sintéticos... (Usando GPU: {use_gpu})")
    synthesizer.fit(df_to_augment)

    # 4. Determinar cuántas muestras generar
    max_class_size = hate_counts.max()
    num_to_generate = max_class_size * len(hate_counts) - hate_counts.sum()

    # 5. Generar y combinar datos
    if num_to_generate > 0:
        print(f"\nGenerando {num_to_generate} muestras sintéticas...")
        df_synthetic = synthesizer.sample(num_rows=num_to_generate)
        df_train_hate_balanced = pd.concat([df_to_augment, df_synthetic], ignore_index=True)
    else:
        df_train_hate_balanced = df_to_augment
else:
    print("\nSolo una sub-categoría presente o no hay datos de odio, no se requiere aumento.")
    df_train_hate_balanced = df_to_augment
    if not df_to_augment.empty:
        # Ajustar el encoder si solo hay una clase
        sub_hate_only_encoder.fit(df_to_augment['sub_label_str'])

print("\nDistribución de sub-categorías DESPUÉS del aumento:")
all_sub_labels = df_to_augment['sub_label_str'].unique()
print(df_train_hate_balanced['sub_label_str'].value_counts().reindex(all_sub_labels, fill_value=0))

# Preparar datos de entrenamiento para el clasificador de sub-categorías
if not df_train_hate_balanced.empty:
    X_train_sub = df_train_hate_balanced[embedding_cols].values
    # Usar el NUEVO encoder para transformar las etiquetas
    y_train_sub = sub_hate_only_encoder.transform(df_train_hate_balanced['sub_label_str'])
else:
    # Crear arrays vacíos si no hay datos para evitar errores posteriores
    X_train_sub = np.array([]).reshape(0, len(embedding_cols))
    y_train_sub = np.array([])

--- Preparando datos para aumento ---
Distribución de sub-categorías ANTES del aumento:
sub_label_str
Race                   321
Sexual Orientation     271
Gender                 158
Physical Appearance     43
Religion                27
Class                   25
Ethnicity               23
Disability              22
Behavior                20
Name: count, dtype: int64

Nuevo encoder para Nivel 2 creado. Clases: ['Behavior' 'Class' 'Disability' 'Ethnicity' 'Gender'
 'Physical Appearance' 'Race' 'Religion' 'Sexual Orientation']

Actualizando metadatos para tratar embeddings como numéricos continuos...
Usando GPU: True

Entrenando CTGAN para generar datos sintéticos... (Usando GPU: True)


c:\Users\emicr\anaconda3\envs\ml_env\Lib\site-packages\sdv\single_table\base.py:162: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
c:\Users\emicr\anaconda3\envs\ml_env\Lib\site-packages\sdv\single_table\base.py:128: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


PerformanceAlert: Using the CTGANSynthesizer on this data is not recommended. To model this data, CTGAN will generate a large number of columns.

Original Column Name   Est # of Columns (CTGAN)
sub_label_str          9
dim_0                  11
dim_1                  11
dim_2                  11
dim_3                  11
dim_4                  11
dim_5                  11
dim_6                  11
dim_7                  11
dim_8                  11
dim_9                  11
dim_10                 11
dim_11                 11
dim_12                 11
dim_13                 11
dim_14                 11
dim_15                 11
dim_16                 11
dim_17                 11
dim_18                 11
dim_19                 11
dim_20                 11
dim_21                 11
dim_22                 11
dim_23                 11
dim_24                 11
dim_25                 11
dim_26                 11
dim_27                 11
dim_28                 11
dim_29                 11
d

## 5. Entrenamiento del Clasificador Principal (Nivel 1) con Optuna y Ensemble

Aquí es donde integramos el pipeline de entrenamiento robusto. Entrenaremos y optimizaremos los cuatro modelos (XGBoost, MLP, y dos Regresiones Logísticas) para la tarea de clasificación binaria (Odio vs. No-Odio). Este proceso no utiliza los datos aumentados, solo el conjunto de entrenamiento original.

In [ ]:
import optuna
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

print("--- Preparando datos para el entrenamiento del Clasificador Principal ---")

# Usar las variables correctas del split jerárquico
y_train = df_train['main_label'].values
y_val = df_val['main_label'].values
num_classes = len(np.unique(y_train)) # Será 2 en este caso

# 1. Escalar características de embeddings
scaler = StandardScaler()
X_train_emb = df_train[embedding_cols].values
X_train_emb_scaled = scaler.fit_transform(X_train_emb)
X_val_emb = df_val[embedding_cols].values
X_val_emb_scaled = scaler.transform(X_val_emb)

# 2. Vectorizar características de texto con TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_train_text = df_train['text_stemmed'].values
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_text)
X_val_text = df_val['text_stemmed'].values
X_val_tfidf = tfidf_vectorizer.transform(X_val_text)
print(f"TF-IDF: {X_train_tfidf.shape[1]} características generadas.")

# 3. Convertir datos de validación a tensores para PyTorch
X_val_torch = torch.tensor(X_val_emb_scaled, dtype=torch.float32).to(device)
y_val_torch = torch.tensor(y_val, dtype=torch.long).to(device)

print("\n✓ Datos escalados, vectorizados y tensores de PyTorch listos para el Nivel 1.")

--- Preparando datos para el entrenamiento del Clasificador Principal ---
TF-IDF: 10000 características generadas.

✓ Datos escalados, vectorizados y tensores de PyTorch listos para el Nivel 1.


In [7]:
# --- 1. Objetivo para MLP con PyTorch (Usa Embeddings) ---
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size, activation_fn, dropout_rate):
        super(MLP, self).__init__()
        layers = []
        current_size = input_size
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(current_size, hidden_size))
            layers.append(activation_fn())
            layers.append(nn.Dropout(dropout_rate))
            current_size = hidden_size
        layers.append(nn.Linear(current_size, output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

def objective_mlp(trial):
    n_layers = trial.suggest_int('n_layers', 1, 3)
    hidden_layers = [trial.suggest_int(f'n_units_l{i}', 32, 256) for i in range(n_layers)]
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop'])
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    activation_fn = getattr(nn, trial.suggest_categorical('activation', ['ReLU', 'Tanh']))
    
    model = MLP(X_train_emb_scaled.shape[1], hidden_layers, num_classes, activation_fn, dropout_rate).to(device)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    train_dataset = TensorDataset(torch.tensor(X_train_emb_scaled, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

    for epoch in range(25): # Menos epochs para HPO más rápido
        model.train()
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            loss = criterion(model(data), target)
            loss.backward()
            optimizer.step()
        
    model.eval()
    with torch.no_grad():
        val_loss = criterion(model(X_val_torch), y_val_torch).item()
    
    trial.report(val_loss, epoch)
    if trial.should_prune(): raise optuna.exceptions.TrialPruned()
    return val_loss

# --- 2. Objetivo para XGBoost (Usa Embeddings) ---
def objective_xgboost(trial):
    params = {
        'objective': 'binary:logistic' if num_classes == 2 else 'multi:softprob',
        'eval_metric': 'logloss' if num_classes == 2 else 'mlogloss',
        'device': 'cuda' if device.type == 'cuda' else 'cpu',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
    }
    model = xgb.XGBClassifier(**params, early_stopping_rounds=10)
    model.fit(X_train_emb, y_train, eval_set=[(X_val_emb, y_val)], verbose=False)
    return log_loss(y_val, model.predict_proba(X_val_emb))

# --- 3. Objetivo para Regresión Logística (Usa Embeddings) ---
def objective_logistic_embeddings(trial):
    params = {'C': trial.suggest_float('C', 1e-4, 1e2, log=True), 'solver': 'liblinear', 'max_iter': 1000}
    model = LogisticRegression(**params, random_state=42)
    model.fit(X_train_emb_scaled, y_train)
    return log_loss(y_val, model.predict_proba(X_val_emb_scaled))

# --- 4. Objetivo para Regresión Logística (Usa TF-IDF) ---
def objective_logistic_tfidf(trial):
    params = {'C': trial.suggest_float('C', 1e-2, 1e2, log=True), 'solver': 'liblinear', 'max_iter': 1000}
    model = LogisticRegression(**params, random_state=42)
    model.fit(X_train_tfidf, y_train)
    return log_loss(y_val, model.predict_proba(X_val_tfidf))

print(f"Funciones objetivo de Optuna para el Nivel 1 definidas.")

Funciones objetivo de Optuna para el Nivel 1 definidas.


In [8]:
models_config = {
    'XGBoost': {'objective_func': objective_xgboost, 'n_trials': 25},
    'MLP_PyTorch': {'objective_func': objective_mlp, 'n_trials': 30},
    'LogisticRegression_Embeddings': {'objective_func': objective_logistic_embeddings, 'n_trials': 20},
    'LogisticRegression_TFIDF': {'objective_func': objective_logistic_tfidf, 'n_trials': 20}
}

model_results = {}

for model_name, config in models_config.items():
    print(f"\n--- Optimizando {model_name} (Nivel 1) ---")
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(config['objective_func'], n_trials=config['n_trials'], show_progress_bar=True)
    
    model_results[model_name] = {
        'best_params': study.best_params,
        'best_score': study.best_value
    }
    print(f"✓ {model_name} completado. Mejor LogLoss: {study.best_value:.4f}")

[I 2025-06-25 01:37:58,209] A new study created in memory with name: no-name-29223502-920d-428b-93bd-b7bf90b6ed07



--- Optimizando XGBoost (Nivel 1) ---


Best trial: 0. Best value: 0.388348:   4%|▍         | 1/25 [00:01<00:45,  1.91s/it]

[I 2025-06-25 01:38:00,114] Trial 0 finished with value: 0.38834796796875687 and parameters: {'n_estimators': 437, 'learning_rate': 0.2536999076681772, 'max_depth': 8}. Best is trial 0 with value: 0.38834796796875687.


Best trial: 1. Best value: 0.330828:   8%|▊         | 2/25 [00:06<01:14,  3.26s/it]

...

Best trial: 7. Best value: 0.183261:  35%|███▌      | 7/20 [00:00<00:00, 35.84it/s]

[I 2025-06-25 01:40:51,885] Trial 0 finished with value: 0.35527615580839433 and parameters: {'C': 0.31489116479568624}. Best is trial 0 with value: 0.35527615580839433.
[I 2025-06-25 01:40:51,922] Trial 1 finished with value: 0.1924042412264549 and parameters: {'C': 63.512210106407046}. Best is trial 1 with value: 0.1924042412264549.
[I 2025-06-25 01:40:51,953] Trial 2 finished with value: 0.18620053322128527 and parameters: {'C': 8.471801418819979}. Best is trial 2 with value: 0.18620053322128527.
[I 2025-06-25 01:40:51,978] Trial 3 finished with value: 0.21830131127974442 and parameters: {'C': 2.481040974867813}. Best is trial 2 with value: 0.18620053322128527.
[I 2025-06-25 01:40:51,992] Trial 4 finished with value: 0.5133252839930367 and parameters: {'C': 0.04207988669606638}. Best is trial 2 with value: 0.18620053322128527.
[I 2025-06-25 01:40:52,006] Trial 5 finished with value: 0.5133373460531051 and parameters: {'C': 0.042070539502879395}. Best is trial 2 with value: 0.1862005

[I 2025-06-25 01:40:52,073] Trial 8 finished with value: 0.21739142624844643 and parameters: {'C': 2.5378155082656657}. Best is trial 7 with value: 0.18326064109811352.
[I 2025-06-25 01:40:52,099] Trial 9 finished with value: 0.18956593831523136 and parameters: {'C': 6.79657809075816}. Best is trial 7 with value: 0.18326064109811352.
[I 2025-06-25 01:40:52,136] Trial 10 finished with value: 0.19640185578666278 and parameters: {'C': 82.29631658321766}. Best is trial 7 with value: 0.18326064109811352.
[I 2025-06-25 01:40:52,169] Trial 11 finished with value: 0.18159010722748264 and parameters: {'C': 20.996451336399733}. Best is trial 11 with value: 0.18159010722748264.
[I 2025-06-25 01:40:52,201] Trial 12 finished with value: 0.18459080131066244 and parameters: {'C': 34.043053954671954}. Best is trial 11 with value: 0.18159010722748264.
[I 2025-06-25 01:40:52,233] Trial 13 finished with value: 0.18140298496485857 and parameters: {'C': 17.14446414712941}. Best is trial 13 with value: 0.18

Best trial: 17. Best value: 0.181396: 100%|██████████| 20/20 [00:00<00:00, 36.50it/s]

[I 2025-06-25 01:40:52,252] Trial 14 finished with value: 0.33732918864876865 and parameters: {'C': 0.3880040916855655}. Best is trial 13 with value: 0.18140298496485857.
[I 2025-06-25 01:40:52,281] Trial 15 finished with value: 0.1836636745703452 and parameters: {'C': 10.66261287572915}. Best is trial 13 with value: 0.18140298496485857.
[I 2025-06-25 01:40:52,304] Trial 16 finished with value: 0.27098679035802226 and parameters: {'C': 0.9253254956089687}. Best is trial 13 with value: 0.18140298496485857.
[I 2025-06-25 01:40:52,334] Trial 17 finished with value: 0.18139642209754364 and parameters: {'C': 18.63934843098346}. Best is trial 17 with value: 0.18139642209754364.
[I 2025-06-25 01:40:52,378] Trial 18 finished with value: 0.20862394501160347 and parameters: {'C': 3.217618717445632}. Best is trial 17 with value: 0.18139642209754364.
[I 2025-06-25 01:40:52,414] Trial 19 finished with value: 0.19892671588852887 and parameters: {'C': 96.04396902719638}. Best is trial 17 with value: 

In [ ]:
main_classifier_models = {} 
print("--- Entrenando modelos finales con los mejores hiperparámetros ---\n")

# 1. XGBoost
params = model_results['XGBoost']['best_params']
final_xgb = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss',
                              device='cuda' if device.type == 'cuda' else 'cpu',
                              **params)
final_xgb.fit(X_train_emb, y_train)
main_classifier_models['XGBoost'] = final_xgb 
print("✓ Modelo XGBoost final entrenado.")

# 2. Regresión Logística (Embeddings)
params = model_results['LogisticRegression_Embeddings']['best_params']
final_log_emb = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000, **params)
final_log_emb.fit(X_train_emb_scaled, y_train)
main_classifier_models['LogisticRegression_Embeddings'] = final_log_emb # Usar el nuevo nombre de variable
print("✓ Modelo Regresión Logística (Embeddings) final entrenado.")

# 3. Regresión Logística (TF-IDF)
params = model_results['LogisticRegression_TFIDF']['best_params']
final_log_tfidf = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000, **params)
final_log_tfidf.fit(X_train_tfidf, y_train)
main_classifier_models['LogisticRegression_TFIDF'] = final_log_tfidf # Usar el nuevo nombre de variable
print("✓ Modelo Regresión Logística (TF-IDF) final entrenado.")

# 4. MLP (PyTorch)
params = model_results['MLP_PyTorch']['best_params']
hidden_layers = [params[f'n_units_l{i}'] for i in range(params['n_layers'])]
final_mlp = MLP(input_size=X_train_emb_scaled.shape[1], hidden_layers=hidden_layers, output_size=num_classes, 
                activation_fn=getattr(nn, params['activation']), dropout_rate=params['dropout_rate']).to(device)
optimizer = getattr(optim, params['optimizer'])(final_mlp.parameters(), lr=params['lr'])
criterion = nn.CrossEntropyLoss()
train_dataset = TensorDataset(torch.tensor(X_train_emb_scaled, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

for epoch in tqdm(range(30), desc="Epochs MLP final"):
    final_mlp.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = final_mlp(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
main_classifier_models['MLP_PyTorch'] = final_mlp.eval() # Usar el nuevo nombre de variable
print("✓ Modelo MLP (PyTorch) final entrenado.")

print("\n✓ Todos los modelos finales han sido entrenados.")

--- Entrenando modelos finales con los mejores hiperparámetros ---

✓ Modelo XGBoost final entrenado.
✓ Modelo Regresión Logística (Embeddings) final entrenado.
✓ Modelo Regresión Logística (TF-IDF) final entrenado.


Epochs MLP final: 100%|██████████| 30/30 [00:01<00:00, 15.77it/s]

✓ Modelo MLP (PyTorch) final entrenado.

✓ Todos los modelos finales han sido entrenados.


In [10]:
print("--- Calculando pesos para el Ensemble del Clasificador Principal (Nivel 1) ---")
val_probas = {}

# Obtener predicciones de cada modelo en el set de validación
val_probas['XGBoost'] = main_classifier_models['XGBoost'].predict_proba(X_val_emb)
val_probas['LogisticRegression_Embeddings'] = main_classifier_models['LogisticRegression_Embeddings'].predict_proba(X_val_emb_scaled)
val_probas['LogisticRegression_TFIDF'] = main_classifier_models['LogisticRegression_TFIDF'].predict_proba(X_val_tfidf)
with torch.no_grad():
    mlp_outputs = main_classifier_models['MLP_PyTorch'](X_val_torch)
    val_probas['MLP_PyTorch'] = torch.softmax(mlp_outputs, dim=1).cpu().numpy()

# Calcular métricas y pesos del ensemble (mayor peso a menor log_loss)
losses = {name: log_loss(y_val, proba) for name, proba in val_probas.items()}
scores = {name: 1.0 / (loss + 1e-9) for name, loss in losses.items()}
total_score = sum(scores.values())
ensemble_weights = {name: score / total_score for name, score in scores.items()}

print("\n--- Pesos del Ensemble de Nivel 1 Calculados ---")
for name, w in ensemble_weights.items(): 
    print(f"{name:<30} | Peso: {w:.3f} | LogLoss (Val): {losses[name]:.4f}")

# También evaluamos aquí el rendimiento en el set de validación para tener una referencia
ensemble_proba_val = np.zeros_like(val_probas['XGBoost'])
for name, proba in val_probas.items():
    ensemble_proba_val += proba * ensemble_weights[name]

ensemble_log_loss_val = log_loss(y_val, ensemble_proba_val)
print(f"\nLogLoss del Ensemble L1 en Validación: {ensemble_log_loss_val:.4f}")

--- Calculando pesos para el Ensemble del Clasificador Principal (Nivel 1) ---

--- Pesos del Ensemble de Nivel 1 Calculados ---
XGBoost                        | Peso: 0.187 | LogLoss (Val): 0.3662
LogisticRegression_Embeddings  | Peso: 0.216 | LogLoss (Val): 0.3174
LogisticRegression_TFIDF       | Peso: 0.377 | LogLoss (Val): 0.1814
MLP_PyTorch                    | Peso: 0.220 | LogLoss (Val): 0.3110

LogLoss del Ensemble L1 en Validación: 0.2174


## 6. Entrenamiento del Clasificador de Sub-categorías (Nivel 2) con Optuna y Ensemble

Ahora, aplicamos la misma metodología robusta al clasificador de Nivel 2. Este se entrenará **únicamente con los datos de 'odio' balanceados sintéticamente**. Crearemos un ensemble de tres modelos (XGBoost, MLP, Regresión Logística) usando los embeddings como características.

In [11]:
print("--- Preparando datos y definiendo objetivos para el Clasificador de Sub-categorías (Nivel 2) ---")

# 1. Dividir los datos de 'odio' aumentados en su propio conjunto de entrenamiento y validación para HPO
if X_train_sub.shape[0] > 0:
    X_sub_train_emb, X_sub_val_emb, y_sub_train, y_sub_val = train_test_split(
        X_train_sub, y_train_sub, test_size=0.25, random_state=42, stratify=y_train_sub
    )

    # 2. Escalar los embeddings para el Nivel 2
    scaler_sub = StandardScaler()
    X_sub_train_emb_scaled = scaler_sub.fit_transform(X_sub_train_emb)
    X_sub_val_emb_scaled = scaler_sub.transform(X_sub_val_emb)

    # 3. Convertir a tensores de PyTorch para el Nivel 2
    X_sub_val_torch = torch.tensor(X_sub_val_emb_scaled, dtype=torch.float32).to(device)
    y_sub_val_torch = torch.tensor(y_sub_val, dtype=torch.long).to(device)
    
    num_sub_classes = len(np.unique(y_train_sub))
    print(f"Datos de Nivel 2 listos. {num_sub_classes} sub-clases detectadas.")

# --- Funciones Objetivo para Optuna (Nivel 2) ---

def objective_xgboost_L2(trial):
    params = {
        'objective': 'multi:softprob', 'num_class': num_sub_classes, 'eval_metric': 'mlogloss',
        'device': 'cuda' if device.type == 'cuda' else 'cpu',
        'n_estimators': trial.suggest_int('n_estimators', 100, 800),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
    }
    model = xgb.XGBClassifier(**params, early_stopping_rounds=10)
    model.fit(X_sub_train_emb, y_sub_train, eval_set=[(X_sub_val_emb, y_sub_val)], verbose=False)
    return log_loss(y_sub_val, model.predict_proba(X_sub_val_emb))

def objective_logistic_L2(trial):
    params = {'C': trial.suggest_float('C', 1e-3, 1e2, log=True), 'solver': 'liblinear', 'max_iter': 1000, 'multi_class': 'ovr'}
    model = LogisticRegression(**params, random_state=42)
    model.fit(X_sub_train_emb_scaled, y_sub_train)
    return log_loss(y_sub_val, model.predict_proba(X_sub_val_emb_scaled))
    
def objective_mlp_L2(trial):
    n_layers = trial.suggest_int('n_layers', 1, 2)
    hidden_layers = [trial.suggest_int(f'n_units_l{i}', 32, 128) for i in range(n_layers)]
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    
    model = MLP(X_train_sub.shape[1], hidden_layers, num_sub_classes, nn.ReLU, 0.3).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    train_dataset = TensorDataset(torch.tensor(X_sub_train_emb_scaled, dtype=torch.float32), torch.tensor(y_sub_train, dtype=torch.long))
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    for epoch in range(20):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            loss = criterion(model(data), target)
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        val_loss = criterion(model(X_sub_val_torch), y_sub_val_torch).item()
    return val_loss

print("Funciones objetivo para Nivel 2 definidas.")

--- Preparando datos y definiendo objetivos para el Clasificador de Sub-categorías (Nivel 2) ---
Datos de Nivel 2 listos. 9 sub-clases detectadas.
Funciones objetivo para Nivel 2 definidas.


In [12]:
if X_train_sub.shape[0] > 0:
    # --- 1. Búsqueda de Hiperparámetros (HPO) para Nivel 2 ---
    models_config_L2 = {
        'XGBoost_L2': {'objective_func': objective_xgboost_L2, 'n_trials': 20},
        'MLP_PyTorch_L2': {'objective_func': objective_mlp_L2, 'n_trials': 25},
        'LogisticRegression_L2': {'objective_func': objective_logistic_L2, 'n_trials': 15}
    }
    model_results_L2 = {}
    for model_name, config in models_config_L2.items():
        print(f"\n--- Optimizando {model_name} (Nivel 2) ---")
        study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
        study.optimize(config['objective_func'], n_trials=config['n_trials'], show_progress_bar=True)
        model_results_L2[model_name] = {'best_params': study.best_params}

    # --- 2. Entrenamiento de los modelos finales del ensemble de Nivel 2 ---
    print("\n--- Entrenando modelos finales del Ensemble (Nivel 2) ---")
    sub_classifier_models = {}
    
    # Entrenar en el conjunto completo de datos de sub-categorías (aumentado)
    X_sub_train_full_scaled = scaler_sub.transform(X_train_sub)
    
    # XGBoost L2
    params = model_results_L2['XGBoost_L2']['best_params']
    final_xgb_L2 = xgb.XGBClassifier(objective='multi:softprob', num_class=num_sub_classes, eval_metric='mlogloss',
                                     device='cuda' if device.type == 'cuda' else 'cpu', **params)
    final_xgb_L2.fit(X_train_sub, y_train_sub)
    sub_classifier_models['XGBoost_L2'] = final_xgb_L2
    
    # Logistic Regression L2
    params = model_results_L2['LogisticRegression_L2']['best_params']
    final_log_L2 = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000, **params)
    final_log_L2.fit(X_sub_train_full_scaled, y_train_sub)
    sub_classifier_models['LogisticRegression_L2'] = final_log_L2
    
    # MLP L2
    params = model_results_L2['MLP_PyTorch_L2']['best_params']
    hidden_layers = [params[f'n_units_l{i}'] for i in range(params['n_layers'])]
    final_mlp_L2 = MLP(X_train_sub.shape[1], hidden_layers, num_sub_classes, nn.ReLU, 0.3).to(device)
    optimizer = optim.Adam(final_mlp_L2.parameters(), lr=params['lr'])
    train_dataset_L2 = TensorDataset(torch.tensor(X_sub_train_full_scaled, dtype=torch.float32), torch.tensor(y_train_sub, dtype=torch.long))
    train_loader_L2 = DataLoader(train_dataset_L2, batch_size=64, shuffle=True)
    for epoch in tqdm(range(30), desc="Epochs MLP L2 final"):
        for data, target in train_loader_L2:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            loss = nn.CrossEntropyLoss()(final_mlp_L2(data), target)
            loss.backward()
            optimizer.step()
    sub_classifier_models['MLP_PyTorch_L2'] = final_mlp_L2.eval()
    print("✓ Todos los modelos del ensemble de Nivel 2 han sido entrenados.")

    # --- 3. Cálculo de pesos para el ensemble de Nivel 2 ---
    print("\n--- Calculando pesos para el Ensemble de Nivel 2 ---")
    val_probas_L2 = {}
    val_probas_L2['XGBoost_L2'] = sub_classifier_models['XGBoost_L2'].predict_proba(X_sub_val_emb)
    val_probas_L2['LogisticRegression_L2'] = sub_classifier_models['LogisticRegression_L2'].predict_proba(X_sub_val_emb_scaled)
    with torch.no_grad():
        mlp_outputs = sub_classifier_models['MLP_PyTorch_L2'](X_sub_val_torch)
        val_probas_L2['MLP_PyTorch_L2'] = torch.softmax(mlp_outputs, dim=1).cpu().numpy()

    losses_L2 = {name: log_loss(y_sub_val, proba, labels=np.unique(y_train_sub)) for name, proba in val_probas_L2.items()}
    scores_L2 = {name: 1.0 / (loss + 1e-9) for name, loss in losses_L2.items()}
    total_score_L2 = sum(scores_L2.values())
    ensemble_weights_L2 = {name: score / total_score_L2 for name, score in scores_L2.items()}

    print("\n--- Pesos del Ensemble de Nivel 2 Calculados ---")
    for name, w in ensemble_weights_L2.items():
        print(f"{name:<25} | Peso: {w:.3f} | LogLoss (Val): {losses_L2[name]:.4f}")
else:
    print("No hay datos para entrenar el clasificador de Nivel 2.")
    sub_classifier_models = None
    ensemble_weights_L2 = None

[I 2025-06-25 01:41:01,204] A new study created in memory with name: no-name-8f0f151a-d40e-40fb-acea-3645c80c84df



--- Optimizando XGBoost_L2 (Nivel 2) ---


Best trial: 0. Best value: 1.8716:   5%|▌         | 1/20 [00:08<02:47,  8.84s/it]

[I 2025-06-25 01:41:10,038] Trial 0 finished with value: 1.871595459867491 and parameters: {'n_estimators': 362, 'learning_rate': 0.17254716573280354, 'max_depth': 7}. Best is trial 0 with value: 1.871595459867491.


Best trial: 1. Best value: 1.86129:  10%|█         | 2/20 [00:23<03:43, 12.43s/it]

...

[I 2025-06-25 01:48:58,050] Trial 13 finished with value: 1.8902156589824692 and parameters: {'C': 0.00864552861056482}. Best is trial 4 with value: 1.886957353270857.


Best trial: 4. Best value: 1.88696: 100%|██████████| 15/15 [01:07<00:00,  4.53s/it]


[I 2025-06-25 01:49:01,232] Trial 14 finished with value: 2.5945114216694423 and parameters: {'C': 0.09683790422339339}. Best is trial 4 with value: 1.886957353270857.

--- Entrenando modelos finales del Ensemble (Nivel 2) ---


c:\Users\emicr\anaconda3\envs\ml_env\Lib\site-packages\sklearn\linear_model\_logistic.py:1288: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
Epochs MLP L2 final: 100%|██████████| 30/30 [00:02<00:00, 11.48it/s]


✓ Todos los modelos del ensemble de Nivel 2 han sido entrenados.

--- Calculando pesos para el Ensemble de Nivel 2 ---

--- Pesos del Ensemble de Nivel 2 Calculados ---
XGBoost_L2                | Peso: 0.575 | LogLoss (Val): 0.4335
LogisticRegression_L2     | Peso: 0.178 | LogLoss (Val): 1.4022
MLP_PyTorch_L2            | Peso: 0.247 | LogLoss (Val): 1.0108


## 7. Evaluación Final del Pipeline Jerárquico Robusto

Evaluamos el pipeline completo. Primero, usamos el **ensemble ponderado de Nivel 1** para la predicción de "Odio vs. No-Odio". Luego, para las predicciones de "odio", usamos el **ensemble ponderado de Nivel 2** para predecir la sub-categoría.

In [ ]:
print("--- Evaluación del pipeline jerárquico en el conjunto de prueba ---")

# 1. Preparar características de prueba para todos los modelos
X_test_emb_eval = df_test[embedding_cols].values
X_test_emb_scaled_eval = scaler.transform(X_test_emb_eval)
X_test_text_eval = df_test['text_stemmed'].values
X_test_tfidf_eval = tfidf_vectorizer.transform(X_test_text_eval)
X_test_torch_eval = torch.tensor(X_test_emb_scaled_eval, dtype=torch.float32).to(device)
y_main_true = df_test['main_label'].values

# 2. Obtener predicciones del ENSEMBLE de Nivel 1
test_probas_L1 = {}
test_probas_L1['XGBoost'] = main_classifier_models['XGBoost'].predict_proba(X_test_emb_eval)
test_probas_L1['LogisticRegression_Embeddings'] = main_classifier_models['LogisticRegression_Embeddings'].predict_proba(X_test_emb_scaled_eval)
test_probas_L1['LogisticRegression_TFIDF'] = main_classifier_models['LogisticRegression_TFIDF'].predict_proba(X_test_tfidf_eval)
with torch.no_grad():
    mlp_outputs = main_classifier_models['MLP_PyTorch'](X_test_torch_eval)
    test_probas_L1['MLP_PyTorch'] = torch.softmax(mlp_outputs, dim=1).cpu().numpy()

final_ensemble_proba_L1 = np.zeros_like(test_probas_L1['XGBoost'])
for name, proba in test_probas_L1.items():
    final_ensemble_proba_L1 += proba * ensemble_weights[name]
y_main_pred = np.argmax(final_ensemble_proba_L1, axis=1)

# 3. Evaluar Nivel 1
print("\n--- [Nivel 1] Rendimiento del Ensemble Principal (Prueba) ---")
print(classification_report(y_main_true, y_main_pred, target_names=['not-hate', 'hate']))

# 4. Obtener y evaluar predicciones del ENSEMBLE de Nivel 2
if sub_classifier_models is not None:
    # Evaluar solo en los datos que son VERDADERAMENTE odio para una métrica justa
    df_test_true_hate = df_test[df_test['main_label'] == 1].copy()
    if not df_test_true_hate.empty:
        # CORRECCIÓN: Usar el nuevo encoder para transformar las etiquetas verdaderas del subconjunto
        y_sub_true = sub_hate_only_encoder.transform(df_test_true_hate['sub_label_str'])
        
        # Preparar datos para el ensemble L2
        X_test_true_hate_emb = df_test_true_hate[embedding_cols].values
        X_test_true_hate_emb_scaled = scaler_sub.transform(X_test_true_hate_emb)
        X_test_true_hate_torch = torch.tensor(X_test_true_hate_emb_scaled, dtype=torch.float32).to(device)
        
        # Obtener y combinar probabilidades L2
        true_hate_probas_L2 = {}
        true_hate_probas_L2['XGBoost_L2'] = sub_classifier_models['XGBoost_L2'].predict_proba(X_test_true_hate_emb)
        true_hate_probas_L2['LogisticRegression_L2'] = sub_classifier_models['LogisticRegression_L2'].predict_proba(X_test_true_hate_emb_scaled)
        with torch.no_grad():
            mlp_outputs_L2 = sub_classifier_models['MLP_PyTorch_L2'](X_test_true_hate_torch)
            true_hate_probas_L2['MLP_PyTorch_L2'] = torch.softmax(mlp_outputs_L2, dim=1).cpu().numpy()
        
        final_true_hate_proba_L2 = np.zeros_like(true_hate_probas_L2['XGBoost_L2'])
        for name, proba in true_hate_probas_L2.items():
            final_true_hate_proba_L2 += proba * ensemble_weights_L2[name]
        y_sub_pred_for_eval = np.argmax(final_true_hate_proba_L2, axis=1)
        
        print("\n--- [Nivel 2] Rendimiento del Ensemble de Sub-categorías (Prueba) ---")
        # Usar las clases del nuevo encoder, que ahora tendrán el tamaño correcto.
        print(classification_report(y_sub_true, y_sub_pred_for_eval, target_names=sub_hate_only_encoder.classes_, zero_division=0))
else:
    print("\nEl clasificador de sub-categorías no fue entrenado.")

--- Evaluación del pipeline jerárquico en el conjunto de prueba ---

--- [Nivel 1] Rendimiento del Ensemble Principal (Prueba) ---
              precision    recall  f1-score   support

    not-hate       0.90      0.98      0.94       833
        hate       0.92      0.72      0.81       303

    accuracy                           0.91      1136
   macro avg       0.91      0.85      0.87      1136
weighted avg       0.91      0.91      0.90      1136


--- [Nivel 2] Rendimiento del Ensemble de Sub-categorías (Prueba) ---
                     precision    recall  f1-score   support

           Behavior       0.00      0.00      0.00         8
              Class       0.67      0.20      0.31        10
         Disability       1.00      0.08      0.15        12
          Ethnicity       0.00      0.00      0.00         9
             Gender       0.51      0.34      0.41        62
Physical Appearance       0.00      0.00      0.00        18
               Race       0.51      0.83   

## 8. Guardado de Artefactos

Guardamos todos los componentes del pipeline jerárquico: los modelos de ambos ensembles, sus respectivos pesos, transformadores y codificadores.

In [14]:
print(f"--- Guardando artefactos en {MODEL_OUTPUT_DIR} ---")

# ... (código para guardar modelos y pesos L1) ...

# 2. Guardar modelos y pesos del ensemble de Nivel 2
if sub_classifier_models is not None:
    # ... (código para guardar modelos y pesos L2) ...
    pass # Asumiendo que esta parte ya está correcta

# 3. Guardar transformadores y codificadores
with open(os.path.join(MODEL_OUTPUT_DIR, "scaler_L1.pkl"), 'wb') as f: pickle.dump(scaler, f)
if 'scaler_sub' in locals():
    with open(os.path.join(MODEL_OUTPUT_DIR, "scaler_L2.pkl"), 'wb') as f: pickle.dump(scaler_sub, f)
with open(os.path.join(MODEL_OUTPUT_DIR, "tfidf_vectorizer.pkl"), 'wb') as f: pickle.dump(tfidf_vectorizer, f)

# CORRECCIÓN: Guardar el nuevo encoder dedicado para las sub-categorías de odio.
if 'sub_hate_only_encoder' in locals():
    with open(os.path.join(MODEL_OUTPUT_DIR, "sub_hate_only_encoder.pkl"), 'wb') as f: pickle.dump(sub_hate_only_encoder, f)
print("\n✓ Scalers, TF-IDF Vectorizer y codificador de sub-etiquetas guardados.")


# 4. Guardar resultados de Optuna
# ... (código para guardar resultados de Optuna) ...

print("\n🎉 Pipeline jerárquico robusto completado y todos los artefactos han sido guardados.")

--- Guardando artefactos en datos_locales\model_output\hierarchical-job-1750836320 ---

✓ Scalers, TF-IDF Vectorizer y codificador de sub-etiquetas guardados.

🎉 Pipeline jerárquico robusto completado y todos los artefactos han sido guardados.
